In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
np.set_printoptions(threshold=np.inf)
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score


import scipy as sp
np.set_printoptions(threshold=np.inf)

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [2]:
# # 读取数据
# dataset= pd.read_excel('data6041.xlsx', sheet_name='Sheet1')
# X = dataset.iloc[:, 1:4]
# y = dataset.iloc[:,4]




In [3]:
# #分测试集和训练集
# from sklearn.model_selection import KFold
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# # 特征归一化
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)



In [4]:
# df=dataset.iloc[:,1:5]
# df_train,df_test=train_test_split(df,test_size = 0.2, random_state = 0)
# df_train.to_csv("d://xunlian.csv")
# df_test.to_csv("d://ceshi.csv")

In [2]:

print("开始读入训练数据...")
df = pd.read_excel('../data/train6041.xlsx', sheet_name='Sheet1')


esc = StandardScaler().fit(df['Fluid resistivity logging'].values.reshape(-1, 1))
fsc = StandardScaler().fit(df['natural gamma-ray logging'].values.reshape(-1, 1))
gsc = StandardScaler().fit(df['spontaneous potential logging'].values.reshape(-1, 1))




e = esc.transform(df['Fluid resistivity logging'].values.reshape(-1, 1))
f = fsc.transform(df['natural gamma-ray logging'].values.reshape(-1, 1))
g = gsc.transform(df['spontaneous potential logging'].values.reshape(-1, 1))



X_train = np.concatenate(( e,f,g), axis=1)
y_train = np.array(df['rock'])



开始读入训练数据...


In [6]:
#生成分类器并训练
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

classifier=AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=8),random_state=0)


In [7]:
# #应用k次交叉验证评价模型性能
# from sklearn.model_selection import cross_val_score
# accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
# accuracies.mean()
# #accuracies.std() 
# #这意味着我们的准确度在（90-100）%之间

In [8]:
#应用网格搜索寻找最佳模型和最佳参数
from sklearn.model_selection import GridSearchCV #对SVC进行检查，网格搜索优化参数。
parameters = [{'n_estimators' : [10,30,50,70,90],'learning_rate' : [0.001,0.01,0.1,1]}] 
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [9]:
#将网格搜索对象拟合到训练集
# grid_search = grid_search.fit(X_train, y_train)

import time
start = time.clock()
grid_search = grid_search.fit(X_train, y_train)
end = time.clock()
print (end-start)

11.902504185902242


In [10]:
#最优精度
best_accuracy = grid_search.best_score_
#最优参数
best_parameters = grid_search.best_params_






In [11]:
print('%.5f' %best_accuracy)
print(best_parameters)


0.80355
{'learning_rate': 0.01, 'n_estimators': 70}


In [12]:
# parameters = [{'n_estimators' : [60,65,70,75,80],'learning_rate' : np.linspace (0.005,0.025,6)}]
parameters = [{'n_estimators' : [60,65,70,75,80],'learning_rate' : [0.005,0.009,0.013,0.017,0.021,0.025]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [13]:
# grid_search = grid_search.fit(X_train, y_train)
import time
start = time.clock()
grid_search = grid_search.fit(X_train, y_train)
end = time.clock()
print (end-start)

21.059127490622522


In [14]:
#最优精度
best_accuracy = grid_search.best_score_
#最优参数
best_parameters = grid_search.best_params_

In [15]:
print('%.5f' %best_accuracy)
print(best_parameters)

0.80642
{'learning_rate': 0.009, 'n_estimators': 75}


In [16]:
parameters = [{'n_estimators' :[60,65,70,75,80],'learning_rate' : [0.009]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [17]:
import time
start = time.clock()
grid_search = grid_search.fit(X_train, y_train)
end = time.clock()
print (end-start)

4.942282872625135


In [19]:
k1 = [60,65,70,75,80]
for x1 in k1:
    classifier= AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=8),learning_rate=0.009, n_estimators= x1,random_state=0)
    classifier.fit(X_train, y_train,)
    accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10 )
   

    print('%.5f' % accuracies.mean())

0.80379
0.80523
0.80476
0.80666
0.80528


In [20]:
x = [60,65,70,75,80]
y=[0.80379,0.80523,0.80476,0.80642,0.80528]
plt.figure(figsize=(8,4),dpi=400)
plt.plot(x,y,color='blue',linewidth=2)
plt.xlabel('Value of n_estimators for model')
plt.ylabel('Cross-Validated accuracy ')
# plt.title('SVM')
plt.ylim(0.803,0.8075)
plt.show
plt.savefig('../Data/AB_accuracy_1.png', format='png')


In [21]:
parameters = [{'n_estimators' : [75],'learning_rate' : [0.005,0.009,0.013,0.017,0.021,0.025]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [22]:
import time
start = time.clock()
grid_search = grid_search.fit(X_train, y_train)
end = time.clock()
print (end-start)

5.977955626157666


In [23]:
k2 = [0.005,0.009,0.013,0.017,0.021,0.025]
for x2 in k2:
    classifier= AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=8),learning_rate=x2, n_estimators= 75,random_state=0)
    classifier.fit(X_train, y_train)
    accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train , cv = 10)
    print('%.5f' % accuracies.mean())

0.80127
0.80666
0.79993
0.80371
0.79779
0.79212


In [24]:
x = [0.005,0.009,0.013,0.017,0.021,0.025]
y=[0.80127,0.80642,0.79993,0.80371,0.79779,0.79212]
plt.figure(figsize=(8,4),dpi=400)
plt.plot(x,y,color='blue',linewidth=2)
plt.xlabel('Value of learning_rate for model')
plt.ylabel('Cross-Validated accuracy ')
# plt.title('SVM')
plt.ylim(0.79,0.81)
plt.show
plt.savefig('../Data/AB_accuracy_2.png', format='png')

In [25]:
#应用网格搜索寻找最佳模型和最佳参数
from sklearn.model_selection import GridSearchCV #对SVC进行检查，网格搜索优化参数。
# parameters = [{'n_estimators' : [71,73,75,77,79],'learning_rate'  : np.linspace (0.008,0.01,11)}]
parameters = [{'n_estimators' : [70,72,74,76,78],'learning_rate'  :[0.008,0.0082,0.0084,0.0086,0.0088,0.009,0.0092,0.0094,0.0096,0.0098,0.01]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [26]:
#将网格搜索对象拟合到训练集
# grid_search = grid_search.fit(X_train, y_train)

import time
start = time.clock()
grid_search = grid_search.fit(X_train, y_train)
end = time.clock()
print (end-start)

38.44816081163029


In [27]:
#最优精度
best_accuracy = grid_search.best_score_
#最优参数
best_parameters = grid_search.best_params_




In [28]:
print('%.5f' %best_accuracy)
print(best_parameters)


0.80690
{'learning_rate': 0.008, 'n_estimators': 74}


In [29]:
parameters = [{'n_estimators' :[70,72,74,76,78],'learning_rate' : [0.008]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [30]:
import time
start = time.clock()
grid_search = grid_search.fit(X_train, y_train)
end = time.clock()
print (end-start)

4.9472001851975165


In [31]:
k3 = [70,72,74,76,78]
for x3 in k3:
    classifier= AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=8),learning_rate=0.008, n_estimators= x3,random_state=0)
    classifier.fit(X_train, y_train,)
    accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train , cv = 10)
   

    print('%.5f' % accuracies.mean())

0.80573
0.80576
0.80718
0.80575
0.80623


In [32]:
x = [70,72,74,76,78]
y=[0.80573,0.80576,0.80690,0.80575,0.80623]
plt.figure(figsize=(8,4),dpi=400)
plt.plot(x,y,color='blue',linewidth=2)
plt.xlabel('Value of n_estimators for model')
plt.ylabel('Cross-Validated accuracy ')
# plt.title('SVM')
plt.ylim(0.804,0.807)

plt.savefig('../Data/AB_accuracy_3.png', format='png')
plt.show

<function matplotlib.pyplot.show>

In [33]:
parameters = [{'n_estimators' : [74],'learning_rate' : [0.008,0.0082,0.0084,0.0086,0.0088,0.009,0.0092,0.0094,0.0096,0.0098,0.01]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [34]:
#将网格搜索对象拟合到训练集
# grid_search = grid_search.fit(X_train, y_train)

import time
start = time.clock()
grid_search = grid_search.fit(X_train, y_train)
end = time.clock()
print (end-start)

9.074598151959435


In [35]:
k4 = [0.008,0.0082,0.0084,0.0086,0.0088,0.009,0.0092,0.0094,0.0096,0.0098,0.01]
for x4 in k4:
    classifier= AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=8),learning_rate=x4, n_estimators= 74,random_state=0)
    classifier.fit(X_train, y_train)
    accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train , cv = 10)
    print('%.5f' % accuracies.mean())

0.80718
0.80378
0.80623
0.80235
0.80330
0.80716
0.80328
0.80193
0.80615
0.80519
0.80374


In [36]:
x = [0.008,0.0082,0.0084,0.0086,0.0088,0.009,0.0092,0.0094,0.0096,0.0098,0.01]
y=[0.80718,0.80378,0.80623,0.80235,0.80330,0.80690,0.80328,0.80193,0.80615,0.80519,0.80374]
plt.figure(figsize=(8,4),dpi=400)
plt.plot(x,y,color='blue',linewidth=2)
plt.xlabel('Value of learning_rate for model')
plt.ylabel('Cross-Validated accuracy ')
# plt.title('SVM')
plt.ylim(0.801,0.81)
plt.show
plt.savefig('../Data/AB_accuracy_4.png', format='png')

In [3]:
from sklearn.model_selection import cross_val_score
classifier= AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=8),learning_rate=0.008, n_estimators= 74)
classifier.fit(X_train, y_train)
# accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train,cv=10 )
# print('训练集精度为：')

# print('%.5f' %accuracies.mean())




AdaBoostClassifier(base_estimator=DecisionTreeClassifier(min_samples_leaf=8,
                                                         min_samples_split=10),
                   learning_rate=0.008, n_estimators=74)

In [4]:

print("正在保存训练数据结果至文件中...")
y_train_pred= classifier.predict(X_train)
y_train_pred = pd.DataFrame({'AB_train_pred': y_train_pred})
Df = pd.concat([df, y_train_pred ], axis=1)
writer = pd.ExcelWriter('../data/train6041.xlsx', engine='xlsxwriter')

Df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()
print("处理完毕，已保存至文件！")
input("Please press <enter>")

print('训练集精度为：')

accuracy_score(y_train,y_train_pred)


正在保存训练数据结果至文件中...
处理完毕，已保存至文件！
Please press <enter>
训练集精度为：


0.9228557738380451

In [5]:
from sklearn.metrics import classification_report
print(classification_report(y_train,y_train_pred,digits=5))

              precision    recall  f1-score   support

           1    1.00000   0.95522   0.97710        67
           2    0.94545   0.74286   0.83200       140
           3    0.94005   0.99402   0.96628       836
           4    0.93126   0.92105   0.92613       456
           5    0.94118   0.66667   0.78049        24
           6    0.87500   0.90960   0.89197       177
           7    0.89109   0.83333   0.86124       108
           8    0.84348   0.81513   0.82906       119
           9    0.93651   0.95161   0.94400        62
          10    0.85714   0.85714   0.85714        98

    accuracy                        0.92286      2087
   macro avg    0.91612   0.86466   0.88654      2087
weighted avg    0.92287   0.92286   0.92136      2087



In [39]:
print("开始读入测试数据...")
testDf = pd.read_excel('../data/test6041.xlsx', sheet_name='Sheet1')


# ttc = csc.transform(testDf[' temperature logging'].values.reshape(-1, 1))
# ttd = dsc.transform(testDf['Vertical Component Logging '].values.reshape(-1, 1))
tte = esc.transform(testDf['Fluid resistivity logging'].values.reshape(-1, 1))
ttf = fsc.transform(testDf['natural gamma-ray logging'].values.reshape(-1, 1))
ttg = gsc.transform(testDf['spontaneous potential logging'].values.reshape(-1, 1))


X_test = np.concatenate((tte,ttf,ttg), axis=1)

y_test = np.array(testDf['rock'])

开始读入测试数据...


In [40]:
classifier.fit(X_test, y_test)
# accuracies = cross_val_score(estimator = classifier, X = X_test, y = y_test)
# print('测试集精度为：')

# print('%.5f' %accuracies.mean())

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=8,
            min_samples_split=10, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=0.008, n_estimators=74, random_state=None)

In [41]:
# y_test_pred = classifier.predict(X_test)
y_test_pred= classifier.predict(X_test)
print("正在保存测试结果至文件中...")
y_test_pred = pd.DataFrame({'AB_test_pred':y_test_pred })
wDf = pd.concat([testDf, y_test_pred ], axis=1)
writer = pd.ExcelWriter('../data/test6041.xlsx', engine='xlsxwriter')

wDf.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()
print("处理完毕，已保存至文件！")
input("Please press <enter>")
print('测试集精度为：')

accuracy_score(y_test,y_test_pred)

正在保存测试结果至文件中...
处理完毕，已保存至文件！
Please press <enter>
测试集精度为：


0.90038314176245215

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred,digits=5))



             precision    recall  f1-score   support

          1    1.00000   0.93750   0.96774        16
          2    0.90909   0.60606   0.72727        33
          3    0.90830   0.99522   0.94977       209
          4    0.87288   0.90351   0.88793       114
          5    1.00000   0.33333   0.50000         3
          6    0.91892   0.85000   0.88312        40
          7    0.86207   0.78125   0.81967        32
          8    0.88462   0.76667   0.82143        30
          9    0.94118   0.88889   0.91429        18
         10    0.89286   0.92593   0.90909        27

avg / total    0.90090   0.90038   0.89638       522



In [43]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
labels = ['1', '2', '3','4','5','6','7','8','9','10']
tick_marks = np.array(range(len(labels))) + 0.5
def plot_confusion_matrix(cm, title='Confusion Matrix', cmap=plt.cm.binary):
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
    plt.title(title)
    plt.colorbar()
    xlocations = np.array(range(len(labels)))
    plt.xticks(xlocations, labels, rotation=90)
    plt.yticks(xlocations, labels)
    plt.ylabel('True label_AB')
    plt.xlabel('Predicted label_AB')
cm = confusion_matrix(y_test,y_test_pred)
np.set_printoptions(precision=3)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(cm_normalized)
plt.figure(figsize=(6, 4), dpi=400)

ind_array = np.arange(len(labels))
x, y = np.meshgrid(ind_array, ind_array)

for x_val, y_val in zip(x.flatten(), y.flatten()):
    c = cm_normalized[y_val][x_val]
    if c >= 0.0:
        plt.text(x_val, y_val, "%0.5f" % (c,), color='red', fontsize=5, va='center', ha='center')
# offset the tick
plt.gca().set_xticks(tick_marks, minor=True)
plt.gca().set_yticks(tick_marks, minor=True)
plt.gca().xaxis.set_ticks_position('none')
plt.gca().yaxis.set_ticks_position('none')
plt.grid(True, which='minor', linestyle='-')
plt.gcf().subplots_adjust(bottom=0.15)

plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix_AdaBoost')
# show confusion matrix
plt.savefig('../Data/confusion_matrix_AB.png', format='png')
# plt.show()

[[ 0.938  0.062  0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.606  0.394  0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.995  0.     0.     0.     0.005  0.     0.     0.   ]
 [ 0.     0.     0.026  0.904  0.     0.009  0.018  0.026  0.009  0.009]
 [ 0.     0.333  0.333  0.     0.333  0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.125  0.     0.85   0.     0.     0.     0.025]
 [ 0.     0.     0.125  0.031  0.     0.031  0.781  0.     0.     0.031]
 [ 0.     0.     0.     0.2    0.     0.033  0.     0.767  0.     0.   ]
 [ 0.     0.     0.     0.111  0.     0.     0.     0.     0.889  0.   ]
 [ 0.     0.     0.     0.037  0.     0.     0.037  0.     0.     0.926]]
